In [ ]:
'''
focus on more advanced concepts such as 
- performance optimization, 
- complex data transformations, 
- large-scale data processing,  
- architectural decision-making. 
'''

### 1. **Optimization of PySpark Jobs**

In [ ]:
# Q) You are processing a large dataset in PySpark that involves multiple joins and aggregations. The job is slow and often times out. 
# What steps would you take to optimize the performance of the PySpark job?
'''
- Enable **predicate pushdown** when reading data.
- Repartition the data intelligently based on join keys.
- Use **broadcast joins** when one of the tables is small.
- Cache intermediate DataFrames if they are used multiple times.
- Avoid using `count()` or `collect()` unless absolutely necessary.
- Set proper **shuffle partitions** based on cluster size (`spark.sql.shuffle.partitions`).
- Use **filter** and **select** to reduce the size of DataFrames early in the process.
'''
# Example: Optimizing joins
small_df = broadcast(small_df)
large_df.join(small_df, 'join_key', 'inner').repartition('join_key').show()


###  Performance and Optimization
'''
Q1) How does PySpark handle large datasets? What optimizations are available for efficient processing?
Q2) Describe how the Catalyst optimizer works in PySpark. How does it optimize logical and physical query plans?

Q3) What are some strategies for optimizing Spark SQL queries in PySpark?
Q4) How would you handle skewed data in PySpark? What techniques can be used to minimize performance degradation?

Q5) How do PySpark partitions work? How can you control the number of partitions in a DataFrame, and why is partitioning 
important for performance?

Q6) What are some ways to manage memory and avoid OutOfMemory errors in PySpark jobs?
Q7) Can you describe a situation where you needed to use custom serialization (Kryo) in PySpark? 
    Why is serialization important, and how does it affect performance?

Q8) You notice that a PySpark job that used to complete in 30 minutes is now taking over 2 hours. 
    How would you troubleshoot and optimize this job?

Q9) What metrics or tools would you use to diagnose performance bottlenecks in Spark?
'''

### 2. **Handling Skewed Data**

In [ ]:
# Q) You notice that one of the partitioned columns in your data has highly skewed data, leading to slow tasks in your Spark job. 
# You are working with two datasets that need to be joined. One of the datasets, `sales_data`, is highly skewed, with 90% of the rows 
# having the same value for the `region` column.
# How would you handle data skewness in PySpark?
'''
- **Salting**: Introduce artificial keys to distribute the data more evenly.
- Use **random partitioning** to reduce the imbalance.
- **Broadcast join** when the smaller dataset is skewed.
- Increase the **shuffle partitions**.
  - Write a solution to handle this skew in PySpark.
       - Hint: Use techniques like **salting** to distribute the skewed data across multiple partitions.
'''  
from pyspark.sql.functions import expr, monotonically_increasing_id

# Example1 of salting
df = df.withColumn("salted_key", expr("concat(join_key, '_', (monotonically_increasing_id() % 10))"))


# Example2: Add salt to distribute skewed data
salted_sales = sales_data.withColumn("salted_region", expr("concat(region, '_', (monotonically_increasing_id() % 10))"))
result = salted_sales.join(other_df, salted_sales['salted_region'] == other_df['region'], "inner")
result.show()


### 3. **Data Pipeline Design**

In [ ]:
#   - **Question:** You are tasked with designing an ETL pipeline to process a daily batch of 1 TB of JSON files stored in S3 and 
# load the processed data into a Redshift table. Explain the design of your pipeline using PySpark and include optimizations for both 
# compute and storage.
'''
     - Use **S3 Select** to filter data while reading large JSON files.
     - Set up **dynamic partition pruning** and **predicate pushdown**.
     - Use **incremental loads** and watermarking to handle late data.
     - Use **DataFrame caching** and repartitioning based on the Redshift schema.
     - Compress the data using Parquet or ORC before loading into Redshift for space and performance efficiency.
     - Use **copy command** or **AWS Glue** for efficient Redshift load.
''' 
df = spark.read.json('s3://bucket/data/')
df_filtered = df.filter(df['date'] == '2024-09-13')  # Partition pruning
df_filtered.write.parquet('/path/to/save')

### 4. **Streaming and Batch Integration**

In [ ]:
# Explain how you would integrate both batch and real-time streaming data pipelines in PySpark. 
# For example, consider you have to process clickstream data (streaming) along with daily user profile updates (batch).
'''
 - Use **Structured Streaming** to handle real-time clickstream data.
 - Batch user profile updates are processed separately and joined with the streaming data.
 - Utilize **watermarking** to handle late data in the streaming pipeline.
 - Output both streams and batch results in the same storage format (e.g., Parquet) for consistency.

'''

# Example: Streaming and batch join
streaming_df = spark.readStream.format("kafka").load()
batch_df = spark.read.parquet("/path/to/batch")
result_df = streaming_df.join(batch_df, "user_id", "left_outer")


### 5. **Advanced Window Functions**

In [ ]:
# You are given a dataset `stock_prices` with columns `symbol`, `date`, and `price`. You are required to calculate the 30-day 
# rolling average price for each stock symbol.
'''
- Write a PySpark solution that computes the rolling average using **Window functions**.
- Use **Window functions** with time-based partitioning.

'''
from pyspark.sql.window import Window
from pyspark.sql.functions import avg

window_spec = Window.partitionBy('symbol').orderBy('date').rowsBetween(-29, 0)
result = stock_prices.withColumn('30_day_avg_price', avg('price').over(window_spec))
result.show()

### 6. **Fault Tolerance and Error Handling in PySpark**

In [ ]:
#- **Question:** Describe how you would handle fault tolerance and ensure data consistency in a large-scale PySpark application 
# that processes real-time sensor data.
'''
- Enable **checkpointing** for long-running streaming jobs to ensure fault tolerance.
- Handle **bad records** using `badRecordsPath` option in file-based data ingestion.
- Implement **idempotent** writes using unique identifiers for each record.
- Use **exactly-once semantics** with Structured Streaming and transactional sinks like Delta Lake.
'''

streaming_df.writeStream \
       .format("delta") \
       .option("checkpointLocation", "/path/to/checkpoint") \
       .start("/path/to/output")


### 7. **Handling Large Data with PySpark and Partitioning Strategies**

In [ ]:
#- **Question:** You have a dataset that exceeds 10 TB, and you need to efficiently store and process it using PySpark. 
# How would you design the storage layout and partitioning strategy to optimize read and write performance?
'''
- Partition the data based on high cardinality columns (e.g., `date`, `region`).
- Use **bucketing** on frequently used join columns to reduce shuffle.
- Store the data in **Parquet/ORC** format with snappy compression for efficient reads and writes.
'''
df.write \
       .partitionBy('date', 'region') \
       .bucketBy(10, 'user_id') \
       .format('parquet') \
       .save('/path/to/save')


### 1. **Large Dataset Join Optimization**
### You have two large datasets: `user_logs` (10 billion rows) and `user_profiles` (10 million rows). 
# The two DataFrames need to be joined on the `user_id` column. The join operation is slow, and you're running into memory issues.
'''
     - **Task:**
       - Write a PySpark solution to optimize this join.
       - Hint: Consider using **broadcast joins** or repartitioning strategies.
'''
from pyspark.sql.functions import broadcast

# Example: Use broadcast join for the smaller dataset
result = user_logs.join(broadcast(user_profiles), "user_id", "inner")
result.show()

### 8. **Custom UDFs and UDAFs**

In [ ]:
# **Question:** Write a custom PySpark UDF that takes a string column and returns the reverse of each string. 
#How would you handle performance issues with UDFs?
'''
     - Use **pandas UDFs** (vectorized UDFs) for performance improvement.
'''
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

@udf(returnType=StringType())
def reverse_string(s):
    return s[::-1]

df.withColumn('reversed', reverse_string(df['column'])).show()

# Pandas UDF
from pyspark.sql.functions import pandas_udf

@pandas_udf(StringType())
def reverse_string_pandas(s):
    return s.apply(lambda x: x[::-1])

df.withColumn('reversed', reverse_string_pandas(df['column'])).show()

### 9. **Cluster Resource Management and Tuning**

In [ ]:
#- **Question:** You are working on a PySpark job in a YARN cluster that frequently runs out of memory. What configurations and tuning 
# steps would you apply to manage memory and resources effectively?
'''
 - Increase **executor memory** and **driver memory** based on the job's requirements.
 - Tune **number of cores** and **executors** to balance parallelism and resource usage.
 - Set **spark.memory.fraction** and **spark.memory.storageFraction** to optimize memory for execution and storage.
 - Use **dynamic allocation** to optimize resource usage across different stages.
'''
spark = SparkSession.builder \
       .config("spark.executor.memory", "8g") \
       .config("spark.executor.cores", "4") \
       .config("spark.dynamicAllocation.enabled", "true") \
       .getOrCreate()

### 4. **Processing Real-Time Streaming Data**

In [ ]:
#You are tasked with processing real-time clickstream data from a Kafka source. Each event contains a `user_id`, `timestamp`, and `action`. 
# You need to write a PySpark Structured Streaming job to:
'''
- Filter out `action = 'login'`
- Group the data by `user_id` and calculate the total number of logins per 5-minute window.
- Output the result to a Parquet file.
- Implement this streaming job using PySpark.
'''
from pyspark.sql.functions import window, col
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("StreamingApp").getOrCreate()

# Read from Kafka source
df = spark.readStream.format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "clickstream") \
  .load()

# Parse the value and create a DataFrame
df_parsed = df.selectExpr("CAST(value AS STRING)").alias("json") \
  .selectExpr("json['user_id']", "json['timestamp']", "json['action']")

# Filter for login actions
df_filtered = df_parsed.filter(col('action') == 'login')

# Group by user and count logins within 5-minute windows
result = df_filtered.groupBy(
  window(col("timestamp"), "5 minutes"), col("user_id")
).count()

# Write the result to Parquet
result.writeStream \
  .outputMode("append") \
  .format("parquet") \
  .option("path", "/path/to/output") \
  .option("checkpointLocation", "/path/to/checkpoint") \
  .start()

### 5. **Handling Large Files with Efficient Partitioning**

In [ ]:
# You have a large dataset (5 TB of Parquet files) stored in S3, and you are tasked with optimizing the reading process for 
# analytical queries on specific columns like `region` and `date`.
'''
 Design an efficient PySpark read operation for this large dataset.
 Apply a partitioning strategy to optimize future reads based on `region` and `date`.
'''
# Read from S3 and partition the dataset by region and date
df = spark.read.parquet("s3://bucket/large_dataset/")
     
# Partition the dataset
df.write.partitionBy("region", "date").parquet("s3://bucket/optimized_dataset/")
     
# Optimize reading by selecting specific partitions
optimized_df = spark.read.parquet("s3://bucket/optimized_dataset/") \
  .filter("region = 'US' AND date = '2024-09-13'")
optimized_df.show()

### 6. **ETL Pipeline for Incremental Data** [ REDSHIFT ]

In [ ]:
# You need to build an ETL pipeline that reads daily JSON files from an S3 bucket, processes the data, 
# and writes it to a Redshift table. The pipeline should only process new data (incremental loads).
'''
 - Implement a PySpark solution to read and process the new files and append the result to Redshift.
 - Ensure that old data is not reprocessed.
'''
 
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()

df = spark.read.json("s3://bucket/daily_data/2024-09-13/")
# Read the latest JSON files
# Apply transformations (e.g., filter, aggregation)
transformed_df = df.filter(df['event'] == 'purchase')
# Write to Redshift
transformed_df.write \
  .format("jdbc") \
  .option("url", "jdbc:redshift://your-redshift-url") \
  .option("dbtable", "public.purchases") \
  .option("user", "username") \
  .option("password", "password") \
  .mode("append") \
  .save()


### 7. **Custom Aggregation with UDAFs**

In [ ]:
# **Problem:** You need to write a custom **User Defined Aggregate Function (UDAF)** in PySpark to calculate the median value for a given 
# column in a DataFrame . Write a PySpark solution using a UDAF to compute the median value of a column.
from pyspark.sql.expressions import UserDefinedAggregateFunction
from pyspark.sql.types import *
class MedianUDAF(UserDefinedAggregateFunction):
    def inputSchema(self):
        return StructType([StructField("input", DoubleType())])
    
    def bufferSchema(self):
        return StructType([StructField("buffer", ArrayType(DoubleType()))])
    
    def dataType(self):
        return DoubleType()
    
    def deterministic(self):
        return True
    
    def initialize(self, buffer):
        buffer[0] = []
    
    def update(self, buffer, input):
        buffer[0].append(input)
    
    def merge(self, buffer1, buffer2):
        buffer1[0].extend(buffer2[0])
    
    def evaluate(self, buffer):
        sorted_values = sorted(buffer[0])
        count = len(sorted_values)
        if count % 2 == 0:
            return (sorted_values[count // 2 - 1] + sorted_values[count // 2]) / 2
        else:
            return sorted_values[count // 2]

# Register and use UDAF
spark.udf.register("median_udaf", MedianUDAF())

df = spark.createDataFrame([(1.0,), (2.0,), (3.0,), (4.0,), (5.0,)], ["value"])
df.selectExpr("median_udaf(value)").show()

### 8. **Managing Data Lineage and Metadata**

In [ ]:
# You need to ensure full data lineage and metadata tracking for your PySpark ETL pipelines.
#Write a PySpark solution to track metadata (e.g., processing time, source file, transformation steps) for every job run and store it in a 
# separate metadata table.

from pyspark.sql.functions import current_timestamp, input_file_name

df = spark.read.csv("s3://bucket/data/")
df = df.withColumn("processing_time", current_timestamp()) \
            .withColumn("source_file", input_file_name())

# Save data and metadata
df.write.parquet("/path/to/processed_data")

metadata_df = df.select("processing_time", "source_file").distinct()
metadata_df.write.mode("append").parquet("/path/to/metadata")


In [ ]:
### Fault Tolerance and Debugging
'''
 Q) How does PySpark handle fault tolerance and recover from failures? Explain the role of lineage and RDDs.
 Q) Describe a scenario where you had to debug a PySpark job. What were the challenges, and how did you 
 approach troubleshooting the issue?
'''

### Memory Management
'''
- A PySpark job is failing with `OutOfMemoryError`. How would you handle this issue in a production environment?
 What changes would you make to avoid this in future?

'''

### Data Skew
'''
- You are joining two large datasets, and the job is running slowly due to skewed data. How would you identify skew, 
and what steps would you take to mitigate this issue?

'''

### Schema Evolution
'''
- In your real-time PySpark pipeline, you need to process data from JSON files, but the schema changes periodically. 
How would you handle schema evolution to ensure the job doesn't break with each change?

'''
### 6. Job Scheduling and Monitoring
'''
- You have a series of PySpark jobs scheduled to run at specific intervals on production. One job failed due to transient network issues. 
How would you ensure the job automatically retries and succeeds?
- What tools or strategies do you use to monitor and alert when PySpark jobs fail?

'''


### 7. Data Partitioning Strategy
'''
- You have a large dataset in AWS S3 that you need to process daily in PySpark. How would you choose the partitioning strategy 
to minimize shuffle and optimize read performance?
- How would you approach re-partitioning the data if it grows significantly over time?
'''

### 8. Data Consistency
'''
- In a real-time streaming application, you encounter issues where duplicate records are being processed. How would you ensure data 
deduplication in a PySpark streaming pipeline?

'''


### 9. Cluster Resource Management
'''
- You are running a PySpark job on a YARN cluster, but the job is failing due to resource allocation issues (e.g., insufficient CPU or memory).
 How would you adjust resource configurations to ensure successful execution?
- How do you manage resource allocation when multiple jobs are running simultaneously on the same cluster?

'''

### 10. Handling Large Joins
'''
- You are working with two massive datasets in PySpark, and a join operation is causing the job to fail due to memory constraints. 
How would you approach this problem?
- How would you decide whether to use a broadcast join or repartition join in this case?

'''

# Question 
'''
Q) How do you monitor the jobs 
Q) Any Problem u have come across in Prod ENV 
   * OutOfMemoryError => Increase the executor Memory

Q) What are some Functional Transformation U have used in Project 

'''